# Import needed libraries

In [ ]:
from statistics import mean,stdev
from termcolor import colored
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import collections
import heapq
import operator
import networkx as nx
import os
import os.path
import random
import numpy as np
from os import walk
from os import path
import sys
from utils import *
from variables import *

# Functions

In [ ]:
def get_top_k_best_nodes(dict_centrality, k):
    first_nodes = heapq.nlargest(k, dict_centrality, key=dict_centrality.get)
    return first_nodes

In [ ]:
def get_bottom_k_best_nodes(dict_centrality, k):
    bottom_nodes = heapq.nsmallest(k, dict_centrality, key=dict_centrality.get)
    return bottom_nodes

In [ ]:
def nth_moment(g,n):
    degree_np = np.array(list(dict(g.degree).values()))
    return (sum(degree_np**n)/len(g))

# IC Simulations for All Networks

In [ ]:
networks = next(walk(RealNetworks), (None, None, []))[2]

for network in networks:
    counter = 0
    g = nx.read_gml(f'{RealNetworks}/{network}',destringizer=int)
    n = network.split(".")[0]

    print(colored(f'Simulation currently on network {network} : {nx.info(g)}...', 'red',attrs=['bold']))
    #Remove Self Loops
    g.remove_edges_from(nx.selfloop_edges(g))

    #Take Largest Component
    nb_components = nx.number_connected_components(g)
    if(nb_components>1):
        subgraphs = list(nx.connected_components(g))
        max = subgraphs[0]
        for subgraph in subgraphs:
            if len(subgraph)>len(max):
                max = subgraph
        g = g.subgraphs(max)
    #print(network)

    #Initialize a dictionary for each centrality
    list_dicts = []

    for centrality in colors.keys():
        globals()[f'dict_{centrality}'] = dict()
        list_dicts.append(globals()[f'dict_{centrality}'])

    #get centrality results
    for dictionary,centrality in zip(list_dicts,colors.keys()):
        p = f'/RealNetworks/centrality_results/{n}/dict_{centrality}.txt'
        with open(p) as raw_data:
            for item in raw_data:
                if ':' in item:
                    key,value = item.split(':', 1)
                    value = value.replace('\n', '')
                    dictionary[int(key)]=float(value)
                else:
                    pass # deal with bad lines of text here
        #print(dictionary)

    #Initialize Average and Standard Deviation for Each Centrality
    for centrality in colors.keys():
        globals()[f'{centrality}_average_each_simulation'] = dict()
        globals()[f'{centrality}_standard_each_simulation'] = dict()
        for fraction in fractions:
            globals()[f'{centrality}_average_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standard_each_simulation'][fraction] = 0

    p = f'./Pickle Variables/{n}/'
    if(path.exists(p)==False):
        os.mkdir(p)
    network_size = len(g.nodes())
    save_obj(network_size,p,'networkSize')

    #Epidemic threshold
    firstMoment = nth_moment(g,1)
    secondMoment = nth_moment(g,2)
    epidemicThreshold = round(firstMoment/(secondMoment-firstMoment),3)
    save_obj(epidemicThreshold,p,'epidemicThreshold')

    #thresholds
    randomThresholds = np.random.uniform(low=0, high=1, size=len(g.nodes()))

    fixedThresholds = [epidemicThreshold+0.1]
    print(fixedThresholds[0])

    #Run diffusion model on centralities
    for counter in range(2):
        for centrality in colors.keys():
            print(colored(f'Counter: {counter}, Centrality: {centrality}', 'green',attrs=['bold']))
            for fraction in fractions:
                globals()[f'l_{centrality}'] = list()
                fraction_infected = round(fraction*network_size)

                if(centrality=="modv_neg"):
                    infected_nodes = get_bottom_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)
                else:
                    infected_nodes = get_top_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)

                for i in range(repeatedSimulations):

                    model = ep.IndependentCascadesModel(g)
                    config = mc.Configuration()

                    if(counter==0):
                        thresholdType = f'Threshold-{fixedThresholds[counter]}'
                        for e in g.edges():
                            config.add_edge_configuration("threshold", e, fixedThresholds[counter])
                    else:
                        thresholdType = 'UniformRandomThresholds'
                        random.shuffle(randomThresholds)
                        for e,threshold in zip(g.edges(),randomThresholds):
                            config.add_edge_configuration("threshold", e, threshold)

                    #config.add_model_parameter('fraction_infected', fraction_infected)
                    config.add_model_initial_configuration('Infected', infected_nodes)
                    model.set_initial_status(config)

                    x = 0
                    iterations = model.iteration_bunch(1)

                    while((iterations[x]['node_count'][1])!=0):
                        x = x + 1
                        iterations = iterations + model.iteration_bunch(1)

                    finalRecoveredInOneIteration = iterations[x]['node_count'][2]
                    #print(finalRecoveredInOneIteration)
                    globals()[f'l_{centrality}'].append(finalRecoveredInOneIteration)

                print(fraction,":",mean(globals()[f'l_{centrality}']))
                globals()[f'{centrality}_average_each_simulation'][fraction] = mean(globals()[f'l_{centrality}'])
                globals()[f'{centrality}_standard_each_simulation'][fraction] = stdev(globals()[f'l_{centrality}'])

            p = f"./Pickle Variables/{n}/{thresholdType}/"
            if(path.exists(p)==False):
                os.mkdir(p)
            save_obj(globals()[f'{centrality}_average_each_simulation'], p ,f'{centrality}_average_each_simulation')
            save_obj(globals()[f'{centrality}_standard_each_simulation'], p , f'{centrality}_standard_each_simulation')

Simulation currently on network board_of_directors.gml : Name: 
Type: Graph
Number of nodes: 854
Number of edges: 2745
Average degree:   6.4286...
0.23500000000000001
Counter: 0, Centrality: modv_pos
0.01 : 250.1
0.02 : 321.89
0.03 : 306.82
0.04 : 330.97
0.05 : 351.76
0.06 : 374.99
0.07 : 373.61
0.08 : 391.03
0.09 : 395.9
0.1 : 403.82
0.11 : 426.34
0.12 : 429.41
0.13 : 444.32
0.14 : 451.32
0.15 : 453.64
0.16 : 468.74
0.17 : 472.03
0.18 : 486.49
0.19 : 489.23
0.2 : 500.73
0.21 : 505.08
0.22 : 512.94
0.23 : 518.57
0.24 : 519.7
0.25 : 522.76
0.26 : 528.04
0.27 : 533.89
0.28 : 545.43
0.29 : 555.62
0.3 : 560.81
0.31 : 570.68
0.32 : 574.4
0.33 : 576.72
0.34 : 580.97
0.35 : 589.68
0.36 : 591.4
0.37 : 597.57
0.38 : 601.15
0.39 : 607.33
0.4 : 615.59
0.41 : 623.83
0.42 : 629.44
0.43 : 640.25
0.44 : 646.62
0.45 : 652.43
0.46 : 663.89
0.47 : 664.23
0.48 : 669.24
0.49 : 671.09
0.5 : 682.96
Counter: 0, Centrality: com
0.01 : 284.06
0.02 : 329.95
0.03 : 378.98
0.04 : 390.07
0.05 : 411.65
0.06 : 427.1

0.49 : 630.99
0.5 : 634.43
Counter: 1, Centrality: modv_pos
0.01 : 31.47
0.02 : 58.2
0.03 : 64.83
0.04 : 80.1
0.05 : 92.38
0.06 : 107.98
0.07 : 120.92
0.08 : 128.78
0.09 : 140.7
0.1 : 153.41
0.11 : 167.51
0.12 : 176.67
0.13 : 195.18
0.14 : 205.05
0.15 : 214.01
0.16 : 229.07
0.17 : 241.89
0.18 : 257.02
0.19 : 263.44
0.2 : 269.05
0.21 : 284.62
0.22 : 297.98
0.23 : 301.69
0.24 : 315.47
0.25 : 321.32
0.26 : 329.66
0.27 : 339.83
0.28 : 351.31
0.29 : 363.9
0.3 : 369.02
0.31 : 381.52
0.32 : 388.36
0.33 : 395.36
0.34 : 402.36
0.35 : 409.68
0.36 : 416.41
0.37 : 425.12
0.38 : 439.15
0.39 : 447.91
0.4 : 456.64
0.41 : 470.42
0.42 : 479.29
0.43 : 490.45
0.44 : 501.22
0.45 : 511.8
0.46 : 520.61
0.47 : 527.5
0.48 : 535.31
0.49 : 543.9
0.5 : 548.64
Counter: 1, Centrality: com
0.01 : 32.15
0.02 : 55.97
0.03 : 86.66
0.04 : 100.41
0.05 : 122.65
0.06 : 135.05
0.07 : 147.95
0.08 : 163.02
0.09 : 177.79
0.1 : 187.39
0.11 : 201.13
0.12 : 214.78
0.13 : 228.53
0.14 : 240.29
0.15 : 248.15
0.16 : 261.83
0.17 : 26

0.01 : 25.25
0.02 : 39.67
0.03 : 51.34
0.04 : 53.1
0.05 : 56.12
0.06 : 60.67
0.07 : 60.26
0.08 : 58.86
0.09 : 63.45
0.1 : 66.12
0.11 : 67.74
0.12 : 67.1
0.13 : 67.87
0.14 : 73.31
0.15 : 70.26
0.16 : 71.39
0.17 : 70.27
0.18 : 72.75
0.19 : 74.32
0.2 : 74.28
0.21 : 74.89
0.22 : 75.25
0.23 : 75.59
0.24 : 74.96
0.25 : 76.72
0.26 : 77.17
0.27 : 77.69
0.28 : 78.57
0.29 : 78.53
0.3 : 80.77
0.31 : 80
0.32 : 82.57
0.33 : 81.51
0.34 : 80.52
0.35 : 82.14
0.36 : 82.59
0.37 : 82.64
0.38 : 83.88
0.39 : 83.17
0.4 : 82.42
0.41 : 83.11
0.42 : 82.74
0.43 : 84.75
0.44 : 85.21
0.45 : 86.05
0.46 : 86.18
0.47 : 87.8
0.48 : 88.03
0.49 : 88.29
0.5 : 89.85
Counter: 0, Centrality: com
0.01 : 31.64
0.02 : 56.13
0.03 : 53.35
0.04 : 56.2
0.05 : 66.68
0.06 : 64.2
0.07 : 65.38
0.08 : 65.98
0.09 : 65.89
0.1 : 68.96
0.11 : 69.94
0.12 : 69.51
0.13 : 72.14
0.14 : 70.99
0.15 : 71.31
0.16 : 71.49
0.17 : 72.55
0.18 : 72.04
0.19 : 74.01
0.2 : 72.43
0.21 : 73.95
0.22 : 74.24
0.23 : 74.78
0.24 : 74.97
0.25 : 75.63
0.26 : 74.17

0.24 : 44.78
0.25 : 45.12
0.26 : 46.56
0.27 : 47.49
0.28 : 49.9
0.29 : 49.39
0.3 : 51.47
0.31 : 53.41
0.32 : 52.64
0.33 : 54.22
0.34 : 54.99
0.35 : 56
0.36 : 58.61
0.37 : 59.17
0.38 : 59.75
0.39 : 61.16
0.4 : 62.86
0.41 : 62.26
0.42 : 63.03
0.43 : 64.37
0.44 : 64.75
0.45 : 65.37
0.46 : 65.65
0.47 : 65.93
0.48 : 66.74
0.49 : 67.33
0.5 : 68.24
Counter: 1, Centrality: cbm
0.01 : 5.09
0.02 : 8.86
0.03 : 10.64
0.04 : 12.75
0.05 : 15.3
0.06 : 17.69
0.07 : 19.95
0.08 : 21.69
0.09 : 23.72
0.1 : 26.45
0.11 : 28.93
0.12 : 32.46
0.13 : 31.82
0.14 : 33.66
0.15 : 33.75
0.16 : 36.12
0.17 : 35.95
0.18 : 36.52
0.19 : 37.86
0.2 : 38.26
0.21 : 40.57
0.22 : 41.64
0.23 : 42.58
0.24 : 44.37
0.25 : 45.91
0.26 : 46.67
0.27 : 47.62
0.28 : 47.82
0.29 : 49.07
0.3 : 51.99
0.31 : 52.74
0.32 : 53.24
0.33 : 54.48
0.34 : 54.94
0.35 : 55.45
0.36 : 57.65
0.37 : 57.8
0.38 : 58.88
0.39 : 59.33
0.4 : 60.7
0.41 : 61.62
0.42 : 62.49
0.43 : 62.68
0.44 : 64.52
0.45 : 65.52
0.46 : 65.31
0.47 : 65.7
0.48 : 67.02
0.49 : 68
0.5 

0.06 : 388.53
0.07 : 393.26
0.08 : 398.01
0.09 : 406.22
0.1 : 413.9
0.11 : 422.16
0.12 : 428.64
0.13 : 437.34
0.14 : 445.51
0.15 : 454.63
0.16 : 465.04
0.17 : 470.66
0.18 : 478.97
0.19 : 489.91
0.2 : 501.46
0.21 : 509.03
0.22 : 520.13
0.23 : 528.11
0.24 : 535.45
0.25 : 547.05
0.26 : 559.49
0.27 : 566.81
0.28 : 579.53
0.29 : 590.81
0.3 : 600.66
0.31 : 609.51
0.32 : 620.17
0.33 : 632.15
0.34 : 644.14
0.35 : 651.69
0.36 : 665.43
0.37 : 676.65
0.38 : 685.38
0.39 : 697.95
0.4 : 709.19
0.41 : 721.92
0.42 : 733.43
0.43 : 743.31
0.44 : 757.64
0.45 : 767.33
0.46 : 778.99
0.47 : 791.43
0.48 : 803.21
0.49 : 814.93
0.5 : 826.41
Counter: 0, Centrality: chb
0.01 : 364.69
0.02 : 363.03
0.03 : 368.3
0.04 : 373.81
0.05 : 382.71
0.06 : 400.28
0.07 : 410.99
0.08 : 423.19
0.09 : 436.21
0.1 : 448.08
0.11 : 463.27
0.12 : 476.09
0.13 : 488.04
0.14 : 499.5
0.15 : 508.41
0.16 : 515.26
0.17 : 522.1
0.18 : 529.76
0.19 : 540.15
0.2 : 545.35
0.21 : 554.7
0.22 : 569.63
0.23 : 582.54
0.24 : 595.61
0.25 : 609.96
0.26

...

0.42 : 504.68
0.43 : 514.45
0.44 : 520.96
0.45 : 528.52
0.46 : 537.11
0.47 : 543.64
0.48 : 555.73
0.49 : 564.71
0.5 : 572.86
Counter: 1, Centrality: cbm
0.01 : 24.39
0.02 : 45.2
0.03 : 67.17
0.04 : 87.64
0.05 : 102.93
0.06 : 121.55
0.07 : 141.27
0.08 : 158.39
0.09 : 173.51
0.1 : 188.82
0.11 : 209.65
0.12 : 223.33
0.13 : 234.92
0.14 : 254.01
0.15 : 264.44
0.16 : 274.23
0.17 : 283.49
0.18 : 292.42
0.19 : 303.87
0.2 : 311.99
0.21 : 323.85
0.22 : 334.83
0.23 : 341.39
0.24 : 349.82
0.25 : 358.88
0.26 : 366.9
0.27 : 379.28
0.28 : 390.77
0.29 : 401.03
0.3 : 412.18
0.31 : 422.05
0.32 : 430.12
0.33 : 440.18
0.34 : 450.19
0.35 : 460.08
0.36 : 468.88
0.37 : 477.85
0.38 : 487.89
0.39 : 497.61
0.4 : 508.65
0.41 : 518.25
0.42 : 526.45
0.43 : 535.9
0.44 : 546.73
0.45 : 554.74
0.46 : 564.09
0.47 : 573.82
0.48 : 583.91
0.49 : 592.55
0.5 : 598.36
Counter: 1, Centrality: chb
0.01 : 16.08
0.02 : 26.08
0.03 : 36.58
0.04 : 46.62
0.05 : 61.28
0.06 : 83.12
0.07 : 92
0.08 : 105.54
0.09 : 115.5
0.1 : 124.83
0.1

0.45 : 261.73
0.46 : 263.36
0.47 : 266.62
0.48 : 267.06
0.49 : 272.23
0.5 : 272.51
Counter: 0, Centrality: cbm
0.01 : 82.79
0.02 : 109.41
0.03 : 124.28
0.04 : 132.04
0.05 : 138.61
0.06 : 147.35
0.07 : 157.4
0.08 : 160.32
0.09 : 162.37
0.1 : 168.03
0.11 : 169.91
0.12 : 173.8
0.13 : 177.16
0.14 : 181.66
0.15 : 183.98
0.16 : 187.56
0.17 : 190.3
0.18 : 194.71
0.19 : 198.82
0.2 : 200.97
0.21 : 204.55
0.22 : 208.19
0.23 : 209.95
0.24 : 212.52
0.25 : 212.53
0.26 : 212.81
0.27 : 215.38
0.28 : 218.58
0.29 : 220.73
0.3 : 223.3
0.31 : 225.91
0.32 : 227.51
0.33 : 229.25
0.34 : 231.18
0.35 : 234.73
0.36 : 235.86
0.37 : 237.22
0.38 : 241.56
0.39 : 241.69
0.4 : 244.3
0.41 : 247.58
0.42 : 252.18
0.43 : 254.46
0.44 : 257.07
0.45 : 260.06
0.46 : 261.64
0.47 : 262.93
0.48 : 266.34
0.49 : 268.21
0.5 : 271.27
Counter: 0, Centrality: chb
0.01 : 85.06
0.02 : 97.58
0.03 : 99.09
0.04 : 103.06
0.05 : 102.26
0.06 : 106.27
0.07 : 106.27
0.08 : 104.81
0.09 : 114.86
0.1 : 132.83
0.11 : 144.66
0.12 : 143.5
0.13 : 14

0.15 : 107.87
0.16 : 112.55
0.17 : 117.1
0.18 : 121.59
0.19 : 127.61
0.2 : 132.61
0.21 : 135.56
0.22 : 139.19
0.23 : 144.69
0.24 : 146.03
0.25 : 154.11
0.26 : 153.51
0.27 : 156.28
0.28 : 157.79
0.29 : 162.8
0.3 : 167.68
0.31 : 168.39
0.32 : 172.96
0.33 : 176.8
0.34 : 180.17
0.35 : 180.53
0.36 : 186.37
0.37 : 189.24
0.38 : 193.7
0.39 : 196.19
0.4 : 200.59
0.41 : 203.78
0.42 : 206.27
0.43 : 212.23
0.44 : 213.74
0.45 : 219.27
0.46 : 220.97
0.47 : 223.31
0.48 : 227.22
0.49 : 230.45
0.5 : 234.24
Counter: 1, Centrality: chb
0.01 : 12.46
0.02 : 22.28
0.03 : 27.6
0.04 : 32.42
0.05 : 35.42
0.06 : 38.93
0.07 : 41.92
0.08 : 45.77
0.09 : 50.8
0.1 : 58.99
0.11 : 67.92
0.12 : 72.19
0.13 : 77.18
0.14 : 81.8
0.15 : 82.08
0.16 : 88.23
0.17 : 92.12
0.18 : 96.91
0.19 : 100.49
0.2 : 104.56
0.21 : 108.07
0.22 : 112.85
0.23 : 116.37
0.24 : 118.45
0.25 : 123.65
0.26 : 127.05
0.27 : 133.06
0.28 : 138.72
0.29 : 142.62
0.3 : 147.62
0.31 : 148.42
0.32 : 150.63
0.33 : 156.56
0.34 : 160.84
0.35 : 163.43
0.36 : 168

0.18 : 818.12
0.19 : 826.82
0.2 : 832.4
0.21 : 838.03
0.22 : 842.12
0.23 : 851.09
0.24 : 857.4
0.25 : 865.13
0.26 : 870.13
0.27 : 883.27
0.28 : 891.3
0.29 : 897.12
0.3 : 903.01
0.31 : 914.84
0.32 : 923.22
0.33 : 932.96
0.34 : 938.97
0.35 : 948.51
0.36 : 960.66
0.37 : 965.11
0.38 : 980.75
0.39 : 987.53
0.4 : 995.03
0.41 : 1007.17
0.42 : 1011.54
0.43 : 1024.44
0.44 : 1040.58
0.45 : 1047.12
0.46 : 1057.7
0.47 : 1067.26
0.48 : 1078.85
0.49 : 1091.72
0.5 : 1102.3
Counter: 0, Centrality: chb
0.01 : 752.11
0.02 : 750.83
0.03 : 754.03
0.04 : 752.75
0.05 : 748.74
0.06 : 756.9
0.07 : 760.09
0.08 : 761.09
0.09 : 766.52
0.1 : 773.02
0.11 : 787.16
0.12 : 801.07
0.13 : 806.24
0.14 : 824.21
0.15 : 832.39
0.16 : 831.04
0.17 : 840.67
0.18 : 852.54
0.19 : 864.05
0.2 : 872.19
0.21 : 871.22
0.22 : 882.4
0.23 : 887.35
0.24 : 888.74
0.25 : 896.47
0.26 : 903.31
0.27 : 913.64
0.28 : 927.49
0.29 : 929.4
0.3 : 937.75
0.31 : 943.93
0.32 : 954.11
0.33 : 962.37
0.34 : 966.4
0.35 : 978.15
0.36 : 983.22
0.37 : 992.3

0.2 : 507.81
0.21 : 521.97
0.22 : 540.07
0.23 : 554.34
0.24 : 568.69
0.25 : 588.31
0.26 : 605.99
0.27 : 623.04
0.28 : 640.37
0.29 : 654.1
0.3 : 672.54
0.31 : 692.04
0.32 : 709.33
0.33 : 720.59
0.34 : 734.82
0.35 : 752.97
0.36 : 770.88
0.37 : 784.59
0.38 : 800.51
0.39 : 816.03
0.4 : 833.26
0.41 : 847.8
0.42 : 865.43
0.43 : 880
0.44 : 895.3
0.45 : 912.09
0.46 : 925.77
0.47 : 943.22
0.48 : 957.48
0.49 : 971.41
0.5 : 984.76
Counter: 1, Centrality: chb
0.01 : 67.66
0.02 : 107.47
0.03 : 140.02
0.04 : 166.12
0.05 : 193.11
0.06 : 217.51
0.07 : 238.26
0.08 : 263.02
0.09 : 288.53
0.1 : 315.99
0.11 : 333.92
0.12 : 355.47
0.13 : 371.73
0.14 : 390.08
0.15 : 404.72
0.16 : 422.03
0.17 : 435.97
0.18 : 455.61
0.19 : 473.36
0.2 : 481.95
0.21 : 499.2
0.22 : 513.29
0.23 : 528.66
0.24 : 544.04
0.25 : 566.93
0.26 : 577.1
0.27 : 598.74
0.28 : 614.26
0.29 : 629.65
0.3 : 646.5
0.31 : 665.27
0.32 : 684.47
0.33 : 701.41
0.34 : 717.5
0.35 : 733.07
0.36 : 748.61
0.37 : 764.34
0.38 : 781.46
0.39 : 799.19
0.4 : 813.